In [1]:
import math
from collections import deque

outer_service_time = 5
safety_coefficient = 1.645
mean = 11
market_std = 7

P_VALUE = 1

class Stage:
    def __init__(self, name, lead_time, added_value, is_leaf):
        self.name = name
        self.lead_time = lead_time
        self.added_value = added_value
        self.holding_cost = added_value
        self.is_leaf = is_leaf

        self.predecessors = []
        self.successors = []

        self.cumulative_time = self.lead_time

        self.temp_degree = 0
        self.p = None
        self.rank = 0

        self.f = {}
        self.g = {}
        self.best_ans = {}

        self.final_S = None
        self.final_SI = None
        self.final_base_stock = None
        
        self.effective_sigma = 0 

    def add_successor(self, stage):
        self.successors.append(stage)
        self.temp_degree += 1
        stage.predecessors.append(self)
        stage.temp_degree += 1

    def calculate_holding_cost(self):
        if self.predecessors:
            for i in self.predecessors:
                self.holding_cost += i.holding_cost

    def calculate_cumulative_time(self):
        if self.predecessors:
            l = [i.cumulative_time for i in self.predecessors]
            self.cumulative_time += max(l)


def label(lis):
    ans = []
    cur = deque()
    for i in lis:
        if i.temp_degree == 1:
            cur.append(i)

    processed = set(cur)
    k = 1
    while cur:
        s = cur.popleft()
        s.rank = k
        k += 1
        for j in s.successors + s.predecessors:
            if j.rank > 0:
                continue
            j.temp_degree -= 1
            s.p = j
            if j.temp_degree == 1:
                if j not in processed:
                    cur.append(j)
                    processed.add(j)
        ans.append(s)
    return ans

def calculate_sigma_propagation(all_nodes, p_val):
    for node in all_nodes:
        get_node_sigma(node, p_val)

def get_node_sigma(node, p_val):
    if node.effective_sigma > 0:
        return node.effective_sigma
    
    if not node.successors: 
        node.effective_sigma = market_std
        return node.effective_sigma
    
    sum_val = 0
    for succ in node.successors:
        s = get_node_sigma(succ, p_val)
        sum_val += s ** p_val
    
    node.effective_sigma = sum_val ** (1.0 / p_val)
    return node.effective_sigma


def ck(stage, S, SI):
    Tk = stage.lead_time
    tau = SI + Tk - S

    if tau < 0:
        return float("inf")

    cost = 0
    cost += stage.holding_cost * safety_coefficient * stage.effective_sigma * math.sqrt(tau)

    for i in stage.predecessors:
        if i.rank < stage.rank:
            # actual_lookup_SI = min(SI, i.cumulative_time)
            # cost += i.f.get(actual_lookup_SI, float("inf"))

            cost += i.f.get(SI, float("inf"))

    for i in stage.successors:
        if i.rank < stage.rank:
            cost += i.g.get(S, float("inf"))

    return cost


def dp(ans):
    for i in range(len(ans) - 1):
        k = ans[i]
        p = ans[i].p
        Mk = k.cumulative_time
        Tk = k.lead_time

        if p in k.successors:  
            for S in range(Mk + 1):
                cost = float("inf")
                best_SI = -1
                lower_SI = max(0, S - Tk)
                upper_SI = Mk - Tk
                for SI in range(lower_SI, upper_SI + 1):
                    val = ck(k, S, SI)
                    if val < cost:
                        cost = val
                        best_SI = SI
                k.f[S] = cost
                k.best_ans[('f', S)] = best_SI
        else:  
            for SI in range(Mk - Tk + 1):
                cost = float("inf")
                best_S = -1
                lower_S = 0
                upper_S = (min(SI + Tk, outer_service_time)
                           if k.is_leaf else SI + Tk)
                for S in range(lower_S, upper_S + 1):
                    val = ck(k, S, SI)
                    if val < cost:
                        cost = val
                        best_S = S
                k.g[SI] = cost
                k.best_ans[('g', SI)] = best_S

    k = ans[-1]
    Mk = k.cumulative_time
    Tk = k.lead_time
    for SI in range(Mk - Tk + 1):
        cost = float("inf")
        best_S = -1
        lower_S = 0
        upper_S = (min(SI + Tk, outer_service_time)
                   if k.is_leaf else SI + Tk)
        for S in range(lower_S, upper_S + 1):
            val = ck(k, S, SI)
            if val < cost:
                cost = val
                best_S = S
        k.g[SI] = cost
        k.best_ans[('g', SI)] = best_S

    return k


def get_optimal_solution(root_node, all_nodes_ordered):
    min_total_cost = float("inf")
    best_root_SI = -1
    for SI, cost in root_node.g.items():
        if cost < min_total_cost:
            min_total_cost = cost
            best_root_SI = SI

    root_node.final_SI = best_root_SI
    root_node.final_S = root_node.best_ans[('g', best_root_SI)]

    reversed_nodes = list(reversed(all_nodes_ordered))

    for k in reversed_nodes[1:]:
        p = k.p
        if p in k.successors:
            requested_S = p.final_SI
            actual_lookup_S = min(requested_S, k.cumulative_time)
            k.final_S = actual_lookup_S

            k.final_SI = k.best_ans[('f', actual_lookup_S)]
            
        elif p in k.predecessors:
            k.final_SI = p.final_S
            k.final_S = k.best_ans[('g', k.final_SI)]

    for k in all_nodes_ordered:
        tau = k.final_SI + k.lead_time - k.final_S
        if tau < 0:
            k.final_base_stock = 0
        else:
            k.final_base_stock = tau * mean + safety_coefficient * k.effective_sigma * math.sqrt(tau)

    return min_total_cost
if __name__ == '__main__':
    camera = Stage('camera', 60, 750, False)
    imager = Stage('imager', 60, 950, False)
    circut_board = Stage('circut_board', 40, 650, False)
    lt_less_60 = Stage('lt_less_60', 60, 150, False)
    lt_more_60 = Stage('lt_more_60', 150, 200, False)
    build = Stage('build', 6, 250, False)
    transfer = Stage('transfer', 2, 50, False)
    ship = Stage('ship', 3, 0, True)

    camera.add_successor(build)
    imager.add_successor(build)
    circut_board.add_successor(build)
    lt_less_60.add_successor(build)
    lt_more_60.add_successor(build)
    build.add_successor(transfer)
    transfer.add_successor(ship)

    lis = [camera, imager, circut_board, lt_less_60, lt_more_60, build, transfer, ship]
    
    for i in lis:
        i.calculate_holding_cost()
        i.calculate_cumulative_time()

    calculate_sigma_propagation(lis, P_VALUE)
    
    print(f"P-Value set to: {P_VALUE}")
    print("Effective Sigma for each node:")
    for i in lis:
        print(f"  Node {i.name}: {i.effective_sigma:.2f}")

    ans = label(lis)
    root_node = dp(ans)
    min_cost = get_optimal_solution(root_node, ans)

    print("-" * 50)
    print(f"Minimum Total Cost: {min_cost:,.2f}")
    print("-" * 50)
    print(f"{'Stage':<10} | {'SI':<6} | {'S':<6} | {'Safety Stock':<12}")
    print("-" * 50)

    for node in lis:
        print(f"{node.name:<10} | {node.final_SI:<6} | {node.final_S:<6} | {node.final_base_stock:<12.2f}")

P-Value set to: 1
Effective Sigma for each node:
  Node camera: 7.00
  Node imager: 7.00
  Node circut_board: 7.00
  Node lt_less_60: 7.00
  Node lt_more_60: 7.00
  Node build: 7.00
  Node transfer: 7.00
  Node ship: 7.00
--------------------------------------------------
Minimum Total Cost: 323,761.31
--------------------------------------------------
Stage      | SI     | S      | Safety Stock
--------------------------------------------------
camera     | 0      | 0      | 749.19      
imager     | 0      | 0      | 749.19      
circut_board | 0      | 0      | 512.83      
lt_less_60 | 0      | 0      | 749.19      
lt_more_60 | 0      | 0      | 1791.03     
build      | 0      | 0      | 94.21       
transfer   | 0      | 2      | 0.00        
ship       | 2      | 5      | 0.00        


In [2]:
import math
from collections import deque

outer_service_time = 5
safety_coefficient = 1.645
mean = 11
market_std = 7

P_VALUE = 1

class Stage:
    def __init__(self, name, lead_time, added_value, is_leaf):
        self.name = name
        self.lead_time = lead_time
        self.added_value = added_value
        self.holding_cost = added_value
        self.is_leaf = is_leaf

        self.predecessors = []
        self.successors = []

        self.cumulative_time = self.lead_time

        self.temp_degree = 0
        self.p = None
        self.rank = 0

        self.f = {}
        self.g = {}
        self.best_ans = {}

        self.final_S = None
        self.final_SI = None
        self.final_base_stock = None
        
        self.effective_sigma = 0 

    def add_successor(self, stage):
        self.successors.append(stage)
        self.temp_degree += 1
        stage.predecessors.append(self)
        stage.temp_degree += 1

    def calculate_holding_cost(self):
        if self.predecessors:
            for i in self.predecessors:
                self.holding_cost += i.holding_cost

    def calculate_cumulative_time(self):
        if self.predecessors:
            l = [i.cumulative_time for i in self.predecessors]
            self.cumulative_time += max(l)


def label(lis):
    ans = []
    cur = deque()
    for i in lis:
        if i.temp_degree == 1:
            cur.append(i)

    processed = set(cur)
    k = 1
    while cur:
        s = cur.popleft()
        s.rank = k
        k += 1
        for j in s.successors + s.predecessors:
            if j.rank > 0:
                continue
            j.temp_degree -= 1
            s.p = j
            if j.temp_degree == 1:
                if j not in processed:
                    cur.append(j)
                    processed.add(j)
        ans.append(s)
    return ans

def calculate_sigma_propagation(all_nodes, p_val):
    for node in all_nodes:
        get_node_sigma(node, p_val)

def get_node_sigma(node, p_val):
    if node.effective_sigma > 0:
        return node.effective_sigma
    
    if not node.successors: 
        node.effective_sigma = market_std
        return node.effective_sigma
    
    sum_val = 0
    for succ in node.successors:
        s = get_node_sigma(succ, p_val)
        sum_val += s ** p_val
    
    node.effective_sigma = sum_val ** (1.0 / p_val)
    return node.effective_sigma


def ck(stage, S, SI):
    Tk = stage.lead_time
    tau = SI + Tk - S

    if tau < 0:
        return float("inf")

    cost = 0
    cost += stage.holding_cost * safety_coefficient * stage.effective_sigma * math.sqrt(tau)

    for i in stage.predecessors:
        if i.rank < stage.rank:
            actual_lookup_SI = min(SI, i.cumulative_time)
            cost += i.f.get(actual_lookup_SI, float("inf"))

            # cost += i.f.get(SI, float("inf"))

    for i in stage.successors:
        if i.rank < stage.rank:
            cost += i.g.get(S, float("inf"))

    return cost


def dp(ans):
    for i in range(len(ans) - 1):
        k = ans[i]
        p = ans[i].p
        Mk = k.cumulative_time
        Tk = k.lead_time

        if p in k.successors:  
            for S in range(Mk + 1):
                cost = float("inf")
                best_SI = -1
                lower_SI = max(0, S - Tk)
                upper_SI = Mk - Tk
                for SI in range(lower_SI, upper_SI + 1):
                    val = ck(k, S, SI)
                    if val < cost:
                        cost = val
                        best_SI = SI
                k.f[S] = cost
                k.best_ans[('f', S)] = best_SI
        else:  
            for SI in range(Mk - Tk + 1):
                cost = float("inf")
                best_S = -1
                lower_S = 0
                upper_S = (min(SI + Tk, outer_service_time)
                           if k.is_leaf else SI + Tk)
                for S in range(lower_S, upper_S + 1):
                    val = ck(k, S, SI)
                    if val < cost:
                        cost = val
                        best_S = S
                k.g[SI] = cost
                k.best_ans[('g', SI)] = best_S

    k = ans[-1]
    Mk = k.cumulative_time
    Tk = k.lead_time
    for SI in range(Mk - Tk + 1):
        cost = float("inf")
        best_S = -1
        lower_S = 0
        upper_S = (min(SI + Tk, outer_service_time)
                   if k.is_leaf else SI + Tk)
        for S in range(lower_S, upper_S + 1):
            val = ck(k, S, SI)
            if val < cost:
                cost = val
                best_S = S
        k.g[SI] = cost
        k.best_ans[('g', SI)] = best_S

    return k


def get_optimal_solution(root_node, all_nodes_ordered):
    min_total_cost = float("inf")
    best_root_SI = -1
    for SI, cost in root_node.g.items():
        if cost < min_total_cost:
            min_total_cost = cost
            best_root_SI = SI

    root_node.final_SI = best_root_SI
    root_node.final_S = root_node.best_ans[('g', best_root_SI)]

    reversed_nodes = list(reversed(all_nodes_ordered))

    for k in reversed_nodes[1:]:
        p = k.p
        if p in k.successors:
            requested_S = p.final_SI
            actual_lookup_S = min(requested_S, k.cumulative_time)
            k.final_S = actual_lookup_S

            k.final_SI = k.best_ans[('f', actual_lookup_S)]
            
        elif p in k.predecessors:
            k.final_SI = p.final_S
            k.final_S = k.best_ans[('g', k.final_SI)]

    for k in all_nodes_ordered:
        tau = k.final_SI + k.lead_time - k.final_S
        if tau < 0:
            k.final_base_stock = 0
        else:
            k.final_base_stock = tau * mean + safety_coefficient * k.effective_sigma * math.sqrt(tau)

    return min_total_cost
if __name__ == '__main__':
    camera = Stage('camera', 60, 750, False)
    imager = Stage('imager', 60, 950, False)
    circut_board = Stage('circut_board', 40, 650, False)
    lt_less_60 = Stage('lt_less_60', 60, 150, False)
    lt_more_60 = Stage('lt_more_60', 150, 200, False)
    build = Stage('build', 6, 250, False)
    transfer = Stage('transfer', 2, 50, False)
    ship = Stage('ship', 3, 0, True)

    camera.add_successor(build)
    imager.add_successor(build)
    circut_board.add_successor(build)
    lt_less_60.add_successor(build)
    lt_more_60.add_successor(build)
    build.add_successor(transfer)
    transfer.add_successor(ship)

    lis = [camera, imager, circut_board, lt_less_60, lt_more_60, build, transfer, ship]
    
    for i in lis:
        i.calculate_holding_cost()
        i.calculate_cumulative_time()

    calculate_sigma_propagation(lis, P_VALUE)
    
    print(f"P-Value set to: {P_VALUE}")
    print("Effective Sigma for each node:")
    for i in lis:
        print(f"  Node {i.name}: {i.effective_sigma:.2f}")

    ans = label(lis)
    root_node = dp(ans)
    min_cost = get_optimal_solution(root_node, ans)

    print("-" * 50)
    print(f"Minimum Total Cost: {min_cost:,.2f}")
    print("-" * 50)
    print(f"{'Stage':<10} | {'SI':<6} | {'S':<6} | {'Safety Stock':<12}")
    print("-" * 50)

    for node in lis:
        print(f"{node.name:<10} | {node.final_SI:<6} | {node.final_S:<6} | {node.final_base_stock:<12.2f}")

P-Value set to: 1
Effective Sigma for each node:
  Node camera: 7.00
  Node imager: 7.00
  Node circut_board: 7.00
  Node lt_less_60: 7.00
  Node lt_more_60: 7.00
  Node build: 7.00
  Node transfer: 7.00
  Node ship: 7.00
--------------------------------------------------
Minimum Total Cost: 297,815.67
--------------------------------------------------
Stage      | SI     | S      | Safety Stock
--------------------------------------------------
camera     | 0      | 60     | 0.00        
imager     | 0      | 60     | 0.00        
circut_board | 0      | 40     | 0.00        
lt_less_60 | 0      | 60     | 0.00        
lt_more_60 | 0      | 60     | 1099.24     
build      | 60     | 0      | 819.55      
transfer   | 0      | 2      | 0.00        
ship       | 2      | 5      | 0.00        
